In [1]:
using JLD2
using Statistics
using DataFrames

In [9]:
algs = ["central"; "central"; "upwind"; "upwind"; "muscl"; "muscl"; "muscl"; "muscl"];
orders = [1; 2; 1; 2; 1; 2; 3; 4];
totalNumGrids = 100;

In [10]:
dataFolderMaxNoise = "$(@__DIR__)/data1DMaxNoise/"  # Noise: dx/2
dataFolderNoise1 = "$(@__DIR__)/data1DNoise1/"  # Noise: dx/2 - 0.05*dx
dataFolderNoise2 = "$(@__DIR__)/data1DNoise2/"  # Noise: dx/2 - 0.1*dx

function loadData(dataFolder)
    m1 = Matrix{Float64}(undef, convert(Int64, length(readdir(dataFolder))/2), length(orders))
    N1 = Vector{Int64}(undef, convert(Int64, length(readdir(dataFolder))/2))
    m2 = Matrix{Float64}(undef, convert(Int64, length(readdir(dataFolder))/2), length(orders))
    N2 = Vector{Int64}(undef, convert(Int64, length(readdir(dataFolder))/2))

    i1 = 1
    i2 = 1
    for file in readdir(dataFolder)
        d = load(dataFolder*file)
        weight = split(file, "_")[end][1:end-5]
        unstableSim = map(x -> x > 0.0 ? true : false, d["res"])
        unstableSims = mapslices(count, unstableSim, dims=1)
        if occursin("exp", file)
            m1[i1, :] .= 100*unstableSims[:]/totalNumGrids
            N1[i1] = d["N"]
            i1 += 1
        else
            m2[i2, :] .= 100*unstableSims[:]/totalNumGrids
            N2[i2] = d["N"]
            i2 += 1
        end
    end

    dfExp = DataFrame(m1, ["$(alg) $(order)" for (alg, order) in zip(algs, orders)]);
    dfExp[!, "N"] = N1;
    sort!(dfExp, :N);
    dfinverse = DataFrame(m2, ["$(alg) $(order)" for (alg, order) in zip(algs, orders)]);
    dfinverse[!, "N"] = N2;
    sort!(dfinverse, :N);
    return dfExp, dfinverse
end

dfExpMax, dfInverseMax = loadData(dataFolderMaxNoise);
dfExpNoise1, dfInverseNoise1 = loadData(dataFolderNoise1);
dfExpNoise2, dfInverseNoise2 = loadData(dataFolderNoise2);

In [11]:
println(dfExpMax)
println(dfInverseMax)

4×9 DataFrame
 Row │ central 1  central 2  upwind 1  upwind 2  muscl 1  muscl 2  muscl 3  muscl 4  N     
     │ Float64    Float64    Float64   Float64   Float64  Float64  Float64  Float64  Int64 
─────┼─────────────────────────────────────────────────────────────────────────────────────
   1 │     100.0      100.0       0.0       0.0    100.0      0.0    100.0      2.0    100
   2 │     100.0      100.0       0.0       0.0    100.0      0.0    100.0      9.0    200
   3 │     100.0      100.0       0.0       0.0    100.0      1.0    100.0     10.0    300
   4 │     100.0      100.0       0.0       0.0    100.0      0.0    100.0     15.0    400
4×9 DataFrame
 Row │ central 1  central 2  upwind 1  upwind 2  muscl 1  muscl 2  muscl 3  muscl 4  N     
     │ Float64    Float64    Float64   Float64   Float64  Float64  Float64  Float64  Int64 
─────┼─────────────────────────────────────────────────────────────────────────────────────
   1 │     100.0      100.0       0.0       0.0     74.0

In [12]:
println(dfExpNoise1)
println(dfInverseNoise1)

4×9 DataFrame
 Row │ central 1  central 2  upwind 1  upwind 2  muscl 1  muscl 2  muscl 3  muscl 4  N     
     │ Float64    Float64    Float64   Float64   Float64  Float64  Float64  Float64  Int64 
─────┼─────────────────────────────────────────────────────────────────────────────────────
   1 │     100.0       99.0       0.0       0.0     99.0      0.0    100.0      1.0    100
   2 │     100.0      100.0       0.0       0.0    100.0      0.0    100.0      1.0    200
   3 │     100.0      100.0       0.0       0.0    100.0      0.0    100.0      2.0    300
   4 │     100.0      100.0       0.0       0.0    100.0      1.0    100.0      3.0    400
4×9 DataFrame
 Row │ central 1  central 2  upwind 1  upwind 2  muscl 1  muscl 2  muscl 3  muscl 4  N     
     │ Float64    Float64    Float64   Float64   Float64  Float64  Float64  Float64  Int64 
─────┼─────────────────────────────────────────────────────────────────────────────────────
   1 │     100.0      100.0       0.0       0.0     47.0

In [13]:
println(dfExpNoise2)
println(dfInverseNoise2)

4×9 DataFrame
 Row │ central 1  central 2  upwind 1  upwind 2  muscl 1  muscl 2  muscl 3  muscl 4  N     
     │ Float64    Float64    Float64   Float64   Float64  Float64  Float64  Float64  Int64 
─────┼─────────────────────────────────────────────────────────────────────────────────────
   1 │     100.0      100.0       0.0       0.0     93.0      0.0    100.0      0.0    100
   2 │     100.0      100.0       0.0       0.0    100.0      0.0    100.0      0.0    200
   3 │     100.0      100.0       0.0       0.0    100.0      0.0    100.0      0.0    300
   4 │     100.0      100.0       0.0       0.0    100.0      0.0    100.0      0.0    400
4×9 DataFrame
 Row │ central 1  central 2  upwind 1  upwind 2  muscl 1  muscl 2  muscl 3  muscl 4  N     
     │ Float64    Float64    Float64   Float64   Float64  Float64  Float64  Float64  Int64 
─────┼─────────────────────────────────────────────────────────────────────────────────────
   1 │     100.0      100.0       0.0       0.0     27.0

In [17]:
file = load("$(@__DIR__)/data1DMaxNoise/gridSensitivity1D_300_expWeight.jld2")
eigs = file["eigs"]

for i = 1:length(algs)
    println("Alg: $(algs[i]), order: $(orders[i]), unstable eigenvalues: $([x for x in eigs[:, i] if x != 0.0])")
end

Alg: central, order: 1, unstable eigenvalues: ComplexF64[13.717215034287875 + 0.0im, 14.136519476692664 + 0.0im, 11.745352338674756 + 0.0im, 2.368596888506418 + 0.0im, 12.236193074276533 + 0.0im, 8.785513791715005 + 0.0im, 9.064564384957126 + 0.0im, 19.932707154664314 + 0.0im, 4.431988744845594 + 0.0im, 3.8780020699680975 + 0.0im, 13.15607379282162 + 0.0im, 8.522528821501256 + 0.0im, 8.878837318107278 + 0.0im, 2.943216630119151 + 0.0im, 4.110959718156549 + 0.0im, 6.280183791977112 + 0.0im, 13.771843735615246 + 0.0im, 3.79018477512307 + 0.0im, 3.703070650857726 + 0.0im, 10.693926575732014 + 0.0im, 8.065639849044222 + 0.0im, 11.525435634230709 + 0.0im, 7.973847980733512 + 0.0im, 14.834611143508432 + 0.0im, 5.361823331305858 + 0.0im, 6.110983545755535 + 0.0im, 4.273164116605846 + 0.0im, 5.131440908235826 + 0.0im, 8.489747562041623 + 0.0im, 6.355001147780081 + 0.0im, 4.32531401084643 + 0.0im, 16.524608188519892 + 0.0im, 11.534954356164551 + 0.0im, 13.730202908545515 + 0.0im, 9.338084494064